In [1]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from midiutil import MIDIFile
import pickle

2024-05-28 05:59:02.011179: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 05:59:02.011230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 05:59:02.012423: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-28 05:59:02.019531: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 05:59:04.593359: W tensorflow/compiler/tf2

In [18]:
def note_to_midi(note, octave):
    # MIDI note numbers for the notes in octave 0
    note_map = {
        'c': 0, 'c#': 1, 'd': 2, 'd#': 3, 'e': 4, 'f': 5, 'f#': 6,
        'g': 7, 'g#': 8, 'a': 9, 'a#': 10, 'b': 11
    }

    # Convert note to lowercase to handle both upper and lower case inputs
    note = note.lower()

    # Calculate the MIDI number
    midi_number = (octave + 1) * 12 + note_map[note]

    return midi_number

def melody_to_midi(melody, rhythm_pattern, velocity_pattern, filename):
    midi = MIDIFile(1)
    midi.addTempo(0, 0, 120)

    for i, note in enumerate(melody):
        pitch_class = note[:-1]
        octave = note[-1]
        pitch = note_to_midi(pitch_class.lower(), int(octave))
        duration = rhythm_pattern[i % len(rhythm_pattern)]
        velocity = velocity_pattern[i % len(velocity_pattern)]
        midi.addNote(0, 0, pitch, i, duration, velocity)  # Add note with duration of 1

    with open(filename, 'wb') as output_file:
        midi.writeFile(output_file)

In [3]:
import os
os.getcwd()
os.chdir("/home/jovyan/workspace/cantus_ai/")

In [27]:
tb_chants = pd.read_csv('output/office_melodies.csv')
tb_chants = tb_chants[tb_chants['mode'].isin([str(x) for x in range(1, 9)])].dropna()
unique_ids = tb_chants.groupby('id')['notes'].apply(list).reset_index().drop_duplicates(subset = 'notes')['id'].tolist()
tb_chants = tb_chants[tb_chants['id'].isin(unique_ids)]
test_set = np.random.choice(unique_ids, 100, replace = False)
train_data = tb_chants[~tb_chants['id'].isin(test_set)]
test_data = tb_chants[tb_chants['id'].isin(test_set)]
melodies = train_data.groupby('id')['notes'].apply(list).tolist()
modes = train_data.groupby('id')['mode'].first().to_list()

In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# Create a vocabulary and mode dictionary
vocab = sorted(set([note for melody in melodies for note in melody]))
vocab_dict = {note: i + 1 for i, note in enumerate(vocab)}  # Start indexing from 1 for padding
vocab_size = len(vocab_dict) + 1  # +1 for padding

mode_dict = {mode: i + 1 for i, mode in enumerate(sorted(set(modes)))}
mode_vocab_size = len(mode_dict) + 1

# Encode melodies and modes using dictionaries
encoded_melodies = [[vocab_dict[note] for note in melody] for melody in melodies]
encoded_modes = [mode_dict[mode] for mode in modes]

# Create position input indicating how many elements are left until the end of the series
positions = [[len(melody) - idx - 1 for idx in range(len(melody))] for melody in encoded_melodies]

# Prepare data for the model
X_series = []
X_modes = []
X_positions = []
y = []

for melody, mode, position in zip(encoded_melodies, encoded_modes, positions):
    for i in range(1, len(melody)):
        X_series.append(melody[:i])
        X_modes.append([mode])
        X_positions.append([position[i-1]])
        y.append(melody[i])

# Pad sequences to have the same length
X_series = tf.keras.preprocessing.sequence.pad_sequences(X_series, padding='pre')

# Convert to numpy arrays
X_series = np.array(X_series)
X_modes = np.array(X_modes)
X_positions = np.array(X_positions)
y = np.array(y)

# Define cosine decay function for learning rate
def cosine_decay(epoch, initial_lr):
    cosine_decay = 0.5 * (1 + np.cos(np.pi * epoch / epochs))
    return initial_lr * cosine_decay

# Parameters
embedding_dim = 24
lstm_units = 32
mode_embedding_dim = 6
dense_units = 64
dropout_rate = 0.2
initial_lr = 0.001
epochs = 50

# Input layers
series_input = Input(shape=(None,), name='series_input')
mode_input = Input(shape=(1,), name='mode_input')
position_input = Input(shape=(1,), name='position_input')

# Embedding layers
series_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='series_embedding')(series_input)
mode_embedding = Embedding(input_dim=mode_vocab_size, output_dim=mode_embedding_dim, name='mode_embedding')(mode_input)

# LSTM layer
lstm1 = LSTM(units=lstm_units, name='lstm1', return_sequences = True, dropout = 0, recurrent_dropout = 0)(series_embedding)
lstm2 = LSTM(units=lstm_units, name='lstm2')(lstm1)

# Concatenate embeddings and position input
concatenated = Concatenate(name='concat_layer')([lstm2, tf.squeeze(mode_embedding, axis=1), position_input])

# Dense layers with dropout and batch normalization
dense1 = Dense(units=dense_units, activation='relu', name='dense1')(concatenated)
dropout1 = Dropout(rate=dropout_rate, name='dropout1')(dense1)
batch_norm1 = BatchNormalization(name='batch_norm1')(dropout1)

dense2 = Dense(units=dense_units, activation='relu', name='dense2')(batch_norm1)
dropout2 = Dropout(rate=dropout_rate, name='dropout2')(dense2)
batch_norm2 = BatchNormalization(name='batch_norm2')(dropout2)


dense3 = Dense(units=dense_units, activation='relu', name='dense3')(batch_norm2)
dropout3 = Dropout(rate=dropout_rate, name='dropout3')(dense3)
batch_norm3 = BatchNormalization(name='batch_norm3')(dropout3)

# Output layer
output = Dense(units=vocab_size, activation='softmax', name='output')(batch_norm3)

# Model
model = Model(inputs=[series_input, mode_input, position_input], outputs=output, name='LSTM_Model')

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(schedule=lambda epoch: cosine_decay(epoch, initial_lr))

# Summary of the model
model.summary()

Model: "LSTM_Model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 series_input (InputLayer)   [(None, None)]               0         []                            
                                                                                                  
 series_embedding (Embeddin  (None, None, 24)             720       ['series_input[0][0]']        
 g)                                                                                               
                                                                                                  
 mode_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 lstm1 (LSTM)                (None, None, 32)             7296      ['series_embedding[0]

In [ ]:
# Fit model
model.fit([X_series, X_modes, X_positions], y,
          validation_split=0.2, epochs=epochs, batch_size=64,
          callbacks=[early_stopping, lr_scheduler])


Epoch 1/50
27721/27721 [==============================] - 475s 17ms/step - loss: 1.4658 - accuracy: 0.4195 - val_loss: 1.3130 - val_accuracy: 0.4913 - lr: 0.0010
Epoch 2/50
27721/27721 [==============================] - 470s 17ms/step - loss: 1.3245 - accuracy: 0.4915 - val_loss: 1.2481 - val_accuracy: 0.5209 - lr: 9.9901e-04
Epoch 3/50
27721/27721 [==============================] - 469s 17ms/step - loss: 1.2753 - accuracy: 0.5155 - val_loss: 1.2146 - val_accuracy: 0.5353 - lr: 9.9606e-04
Epoch 4/50
27721/27721 [==============================] - 470s 17ms/step - loss: 1.2490 - accuracy: 0.5262 - val_loss: 1.2004 - val_accuracy: 0.5403 - lr: 9.9114e-04
Epoch 5/50
27721/27721 [==============================] - 469s 17ms/step - loss: 1.2311 - accuracy: 0.5332 - val_loss: 1.1895 - val_accuracy: 0.5427 - lr: 9.8429e-04
Epoch 6/50
27721/27721 [==============================] - 469s 17ms/step - loss: 1.2195 - accuracy: 0.5376 - val_loss: 1.1842 - val_accuracy: 0.5454 - lr: 9.7553e-04
Epoch 7/

In [ ]:
# Save the model
model.save('models/office/melody_prediction_model_with_notes_until_end.h5')

# Save the dictionaries
with open('models/office/pitch_encoder.pkl', 'wb') as f:
    pickle.dump(vocab_dict, f)

with open('models/office/mode_encoder.pkl', 'wb') as f:
    pickle.dump(mode_dict, f)

In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

# Function to load the model and dictionaries
def load_resources(model_path, vocab_dict_path, mode_dict_path):
    model = load_model(model_path)
    with open(vocab_dict_path, 'rb') as f:
        vocab_dict = pickle.load(f)
    with open(mode_dict_path, 'rb') as f:
        mode_dict = pickle.load(f)
    return model, vocab_dict, mode_dict

# Function to predict the element probabilities for a given sequence, mode, and remaining notes
def predict_element_probabilities(sequence, mode, notes_left, model, vocab_dict, mode_dict):
    # Reverse the vocab_dict to get the reverse mapping from indices to elements
    index_to_vocab = {index: element for element, index in vocab_dict.items()}

    # Encode the sequence and mode using the dictionaries
    encoded_sequence = [vocab_dict[element] for element in sequence]
    encoded_mode = np.array([mode_dict[mode]]).reshape(1, 1)

    # Create the position input based on the notes left in the series
    position_input = np.array([[notes_left]])

    # Pad the sequence to match the input shape of the model
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences([encoded_sequence], padding='pre')

    # Predict the probabilities
    probabilities = model.predict([padded_sequence, encoded_mode, position_input])[0]

    # Map the probabilities back to the elements
    element_probabilities = {index_to_vocab[i]: prob for i, prob in enumerate(probabilities) if i in index_to_vocab}

    return element_probabilities

# Function to extend a sequence to a specified length using temperature-controlled sampling
def extend_sequence(initial_sequence, mode, target_length, temperature, model, vocab_dict, mode_dict):
    sequence = initial_sequence[:]
    while len(sequence) < target_length:
        notes_left = target_length - len(sequence)
        element_probabilities = predict_element_probabilities(sequence, mode, notes_left, model, vocab_dict, mode_dict)

        # Sort elements by probability
        sorted_elements = sorted(element_probabilities.items(), key=lambda item: item[1], reverse=True)

        # Determine the number of elements to sample from based on temperature
        num_elements = int(len(sorted_elements) * temperature)
        num_elements = max(1, num_elements)  # Ensure at least one element is sampled

        # Sample the next element based on the adjusted probabilities
        elements, probabilities = zip(*sorted_elements[:num_elements])
        probabilities = np.array(probabilities) / np.sum(probabilities)  # Normalize probabilities
        next_element = np.random.choice(elements, p=probabilities)

        # Append the next element to the sequence
        sequence.append(next_element)

    return sequence

# Load resources
model, vocab_dict, mode_dict = load_resources('models/mass/melody_prediction_model_with_notes_until_end.h5',
                                              'models/mass/pitch_encoder.pkl',
                                              'models/mass/mode_encoder.pkl')

# Example usage
initial_sequence = ["F4", "E4"]
mode = "2"
target_length = 16
temperature = 0.5  # Specify the temperature value
extended_sequence = extend_sequence(initial_sequence, mode, target_length, temperature, model, vocab_dict, mode_dict)

print("Extended Sequence:", extended_sequence)


1/1 [==============================] - 0s 17ms/step
Extended Sequence: ['F4', 'E4', 'D4', 'D4', 'G4', 'F4', 'F4', 'G4', 'F4', 'A4', 'G4', 'F4', 'D4', 'D4', 'F4', 'F4']


In [24]:
melody_to_midi(extended_sequence,[1, 1, 1], [127, 70, 100, 50], "generated_melody.mid")